In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import time

from api import *
from dataframe import *
from database import *
from file import *
from sql import *
from tables import *

ModuleNotFoundError: No module named 'matplotlib'

# cfg

In [ ]:
cfg = {
    "host": f"{os.environ['POSTGRE_HOST']}",
    "user": f"{os.environ['POSTGRE_USER']}",
    "password": f"{os.environ['POSTGRE_PASSWORD']}",
    "port": "5432",
    "database": "herbario"
}
filename = "original.csv"
filename_george = "dados-george.csv"

engine, session = connect(cfg)

# Load data

In [ ]:
dataframe = pandas.read_csv("original.csv", sep=";", low_memory=False, skipinitialspace=True)
dataframe_george = pandas.read_csv(filename_george, sep=";", low_memory=False, skipinitialspace=True)

In [ ]:
create_table_if_not_exists(cfg, engine, "county")
create_table_if_not_exists(cfg, engine, "data")

In [ ]:
if session.query(County).count() == 0:
    response = get_municipies()
    for i, county in enumerate(response.json()):
        session.add(create_county(county))
        make_operation(session)

In [ ]:
if session.query(DataSP).count() == 0:
    change_header(dataframe)
    dataframe = preprocess(dataframe)
    # dataframe.iloc[24230]
    for row in dataframe.iterrows():
        session.add(create_datasp(row[1]))
        make_operation(session)

In [ ]:
if session.query(DataSP).filter(DataSP.george == True).count() == 0:
    for row in dataframe.iterrows():
        if row[1]["GEORGE"].lower() == "sim":
            session.query(DataSP).filter(DataSP.seq == row[1]["seq"]).update({"george": True}, synchronize_session=False)
            make_operation(session)

In [ ]:
data_piperaceae = session.query(DataSP).all()
data_uf = session.query(sqlalchemy.func.lower(unaccent(County.uf))).distinct().all()
data_uf_name = session.query(County.uf_name, sqlalchemy.func.lower(unaccent(County.uf_name))).distinct().all()
data_county = session.query(County.county, unaccent(County.county)).filter(County.uf=="PR").all()

In [ ]:
# 1
# len(q)

CREATE INDEX persjon_idx ON person USING GIN (to_tsvector('simple', name));

# CREATE INDEX local_idx ON data USING
# gin(
#     (setweight(to_tsvector('portuguese', country),'A') ||  setweight(to_tsvector('portuguese', county), 'B') ||  setweight(to_tsvector('portuguese', state_province), 'C'))
# );

# session.query(DataSP).filter(DataSP.__ts_vector__.match("", postgresql_regconfig='portuguese')).all()

# SELECT
#   ts_rank(
#     setweight(to_tsvector('portuguese', 'maringa'), 'A') || setweight(to_tsvector('portuguese', 'maringa'), 'B'),
#     to_tsquery('portuguese', 'maringa')
#   ) AS elephant_rank;
#

# session.query(DataSP, sqlalchemy.func.ts_rank(sqlalchemy.func.setweight(sqlalchemy.func.to_tsvector('portuguese', 'county'), 'A'), sqlalchemy.func.to_tsvector('portuguese', 'maringa'))).all()

In [28]:
list_seq = list([])
list_of_columns_valid = get_columns_text()

In [ ]:
start = time.process_time()

for d in data_country:
    find_and_update(d, list_seq, "my_country", value="Brasil")

end = time.process_time()
elapsed_time = end-start
print(f"elapsed_time: {elapsed_time} seconds")

In [ ]:
start = time.process_time()

for (original, preprocessed) in data_uf_name:
    find_and_update(preprocessed, list_seq, "my_state", original)

end = time.process_time()
elapsed_time = end-start
print(f"elapsed_time: {elapsed_time} seconds")

In [ ]:
start = time.process_time()

for (original, preprocessed) in data_county:
    find_and_update(preprocessed, list_seq, "my_city", original)

end = time.process_time()
elapsed_time = end-start
print(f"elapsed_time: {elapsed_time} seconds")

In [14]:
for l in list_seq:
    for y in l["seq"]:
        session.query(DataSP).filter(DataSP.seq == y).update(values=dict({l["new_column"]: l["value_searched"]}), synchronize_session=False)
        make_operation(session)

In [ ]:
create_outfile(list_seq)

In [ ]:
data_has_only_state = session.query(DataSP.seq, DataSP.my_state).filter(sqlalchemy.and_(DataSP.my_state.is_not(None), DataSP.my_country.is_(None))).all()
make_operation(session)
len(data_has_only_state)

In [ ]:
for (seq, state) in sorted(data_has_only_state):
    session.query(DataSP).filter(DataSP.seq == seq).update(values=dict({"my_country": "Brasil"}), synchronize_session=False)
    make_operation(session)

In [ ]:
def get_state_of_city(city):
    state_of_city = session.query(County.uf_name).filter(County.county == city).all()
    make_operation(session)
    return state_of_city

data_has_only_county = session.query(DataSP.seq, DataSP.my_city).filter(sqlalchemy.and_(DataSP.my_city.is_not(None), DataSP.my_state.is_(None), DataSP.my_country.is_(None))).all()
make_operation(session)

len(data_has_only_county)

In [ ]:
for (seq, city) in data_has_only_county:
    state_of_city = get_state_of_city(city)
    session.query(DataSP).filter(DataSP.seq == seq).update(values=dict({"my_state": state_of_city, "my_country": "Brasil"}), synchronize_session=False)
    make_operation(session)

In [ ]:
session.close()
engine.dispose()